# Natural Language Processing with Disaster Tweets

### Team: Robyn Tomson and Otto-Cristofer Vanasaun

#### Downloading and importing essential dependencies and Python packages.

In [22]:
!pip install transformers datasets torch scikit-learn
!pip install tf-keras
!pip install tensorflow
!pip install transformers
!pip install --upgrade tensorflow transformers
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from collections import Counter
import torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 13.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


#### Disabling warnings

In [2]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

#### Reading data from csv files

In [3]:
dataTrain = pd.read_csv("data/train.csv")
dataTest = pd.read_csv("data/test.csv")
dataSampleSubmit = pd.read_csv("data/sample_submission.csv")

#### Data description and understanding

In [4]:
disasters = []
total_words = Counter()
for row in dataTrain.itertuples(index=True):
    if pd.notna(row.keyword): 
        if row.keyword not in disasters:
            disasters.append(row.keyword)
            
    words = row.text.split() 
    total_words.update(words)
    
unique_word_count = len(total_words)


print(dataTrain.info())
print("________________________________________")
print("Total words:", sum(total_words.values()))
print("Unique words:", unique_word_count)
print("________________________________________")
print("Number of unique disasters:", len(disasters))
print("List of unique disasters:", disasters)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
None
________________________________________
Total words: 113461
Unique words: 31924
________________________________________
Number of unique disasters: 221
List of unique disasters: ['ablaze', 'accident', 'aftershock', 'airplane%20accident', 'ambulance', 'annihilated', 'annihilation', 'apocalypse', 'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked', 'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze', 'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood', 'bloody', 'blown%20up', 'body%20bag', 'body%20bagging', 'body%20bags', 'bomb', '

#### Data preparation

In [45]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
max_len = 1400
train_sequences = pad_sequences(
    dataTrain['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=max_len, truncation=True)),
    maxlen=max_len, padding='post'
)


train_sequences, val_sequences, train_labels, val_labels = train_test_split(
    train_sequences, dataTrain['label'], test_size=0.2, random_state=42
)

def preprocess_data(data):
    label_encoder = LabelEncoder()
    data['label'] = label_encoder.fit_transform(data['target'])

    data['text'] = data['text']

    return data, label_encoder

def tokenize_and_prepare(data, max_length=512):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


    data['tokens'] = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=max_length, truncation=True))
    token_sequences = pad_sequences(data['tokens'], maxlen=max_length, padding='post')

    return token_sequences


def eeltöötlus(trainset):

    trainset, label_encoder = preprocess_data(trainset)
    #testset, _ = preprocess_data(testset)

    train_data, val_data = train_test_split(trainset, test_size=0.2, random_state=42)

    train_sequences = tokenize_and_prepare(train_data)
    val_sequences = tokenize_and_prepare(val_data)
    #test_sequences = tokenize_and_prepare(testset)

    return train_sequences, val_sequences, label_encoder

train_sequences, val_sequences, label_encoder = eeltöötlus(dataTrain)

#### Model building

In [50]:
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from time import time
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import pandas as pd

import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from sklearn.utils.class_weight import compute_class_weight

from transformers import BertTokenizer, TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight



vocab_size = 30522
embedding_dim = 128
max_len = 1000

modelLSTM = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    SpatialDropout1D(0.2),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(5, activation='softmax')
])


modelLSTM.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#unique_classes = np.unique(train_labels)
#class_weights = compute_class_weight('balanced', classes=unique_classes, y=train_labels)
#class_weights_dict = dict(zip(unique_classes, class_weights))

algus = time()

try:
    history = modelLSTM.fit(
        train_sequences,
        train_labels,
        epochs=12,
        batch_size=32,
        validation_data=(val_sequences, val_labels),
        #class_weight=class_weights_dict,
        verbose=1
    )
except KeyError as e:
    print("KeyError encountered:", e)


Epoch 1/12
191/191 ━━━━━━━━━━━━━━━━━━━━ 41s 197ms/step - accuracy: 0.5208 - loss: 0.9017 - val_accuracy: 0.5739 - val_loss: 0.6858
Epoch 2/12
191/191 ━━━━━━━━━━━━━━━━━━━━ 38s 198ms/step - accuracy: 0.5370 - loss: 0.7025 - val_accuracy: 0.5739 - val_loss: 0.6830
Epoch 3/12
191/191 ━━━━━━━━━━━━━━━━━━━━ 38s 199ms/step - accuracy: 0.5398 - loss: 0.6937 - val_accuracy: 0.5739 - val_loss: 0.6827
Epoch 4/12
191/191 ━━━━━━━━━━━━━━━━━━━━ 44s 230ms/step - accuracy: 0.5727 - loss: 0.6855 - val_accuracy: 0.5739 - val_loss: 0.6878
Epoch 5/12
191/191 ━━━━━━━━━━━━━━━━━━━━ 69s 362ms/step - accuracy: 0.5371 - loss: 0.6913 - val_accuracy: 0.5739 - val_loss: 0.6827
Epoch 6/12
191/191 ━━━━━━━━━━━━━━━━━━━━ 63s 331ms/step - accuracy: 0.5724 - loss: 0.6863 - val_accuracy: 0.5739 - val_loss: 0.6832
Epoch 7/12
191/191 ━━━━━━━━━━━━━━━━━━━━ 47s 248ms/step - accuracy: 0.5701 - loss: 0.6850 - val_accuracy: 0.5739 - val_loss: 0.6825
Epoch 8/12
191/191 ━━━━━━━━━━━━━━━━━━━━ 41s 215ms/step - accuracy: 0.5743 - loss: 0

In [51]:
aeg = time()-algus
print("Aega treenimiseks läks {} minutit ja {} sekundit.".format(round(aeg/60), round(aeg%60,2)))
"""
X_testLSTM = test_sequences

y_predsLSTM = modelLSTM.predict(X_testLSTM)

y_predsLSTM = [np.argmax(vastus) for vastus in y_predsLSTM]

y_test = list(testset['label'])
#y_test = [labels.index(cat) for cat in y_test]

print("Täpsus:", accuracy_score(y_test, y_predsLSTM))


print("Sildid:", labels)
print("Segadusmaatriks:")
print(confusion_matrix(y_test, y_predsLSTM))
"""

Aega treenimiseks läks 10 minutit ja 51.17 sekundit.


'\nX_testLSTM = test_sequences\n\ny_predsLSTM = modelLSTM.predict(X_testLSTM)\n\ny_predsLSTM = [np.argmax(vastus) for vastus in y_predsLSTM]\n\ny_test = list(testset[\'label\'])\n#y_test = [labels.index(cat) for cat in y_test]\n\nprint("Täpsus:", accuracy_score(y_test, y_predsLSTM))\n\n\nprint("Sildid:", labels)\nprint("Segadusmaatriks:")\nprint(confusion_matrix(y_test, y_predsLSTM))\n'

#### Model training

In [ ]:
#TODO

#### Model testing

In [ ]:
#TODO

#### Submition, review, reporting and ...

In [ ]:
#TODO